Word2Vec -  слишком медленное получение векторов - решено не делать

# KNN классификация на основе tfidf

In [11]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, ndcg_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

## Prepare train test

In [80]:
load_clean_data = True

if load_clean_data:
    train = pd.read_csv('/kaggle/input/cl-cup-it-2023-embs/train_data_result.csv')
    test = pd.read_csv('/kaggle/input/cl-cup-it-2023-embs/test_data_result.csv')
else:
    train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)
    test = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

In [83]:
def prepare_df(train, test=False):
    train = train.explode('comments')
    if not test:
        train['score'] = train['comments'].str['score']
    train['comment'] = train['comments'].str['text']
    train = train.drop(columns=['comments'])
    train['text_id'] = train.index
    return train

def prepare_df_cleaned(train):
    train = train.rename(columns={'post_text':'text'})
    train['text_id'] = train['text'].factorize()[0]
    train['comment'] = train['comment'].fillna('')
    
    return train

if not load_clean_data:
    train = prepare_df(train)
    scores = train['score'].values
    test = prepare_df(test, True)
else:
    train = prepare_df_cleaned(train)
    test = prepare_df_cleaned(test)
    
train.head(20)

,text,comment,score,text_id
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0.0,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who dont see th...,1.0,0
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2.0,0
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3.0,0
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4.0,0
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0.0,1
6,CBS acquires last.fm for $280m,Does this mean that theres now a bigname compa...,1.0,1
7,CBS acquires last.fm for $280m,Also on BBC News httpnews.bbc.co.uklowtechnolo...,2.0,1
8,CBS acquires last.fm for $280m,I dont understand what they do that is worth M...,3.0,1
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,4.0,1


## Word2Vec

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess    

def tidy_sentence(sentence, vocabulary):
    return [word for word in simple_preprocess(sentence) if word in vocabulary]    

def compute_sentence_similarity(sentence_1, sentence_2, model_wv):
    vocabulary = set(model_wv.index_to_key)    
    tokens_1 = tidy_sentence(sentence_1, vocabulary)    
    tokens_2 = tidy_sentence(sentence_2, vocabulary)    
    return model_wv.n_similarity(tokens_1, tokens_2)

sim = compute_sentence_similarity('this is a sentence', 'this is also a sentence', model)
print(sim)

### Get Word2Vec vectors for given sentence:

In [ ]:
sentence = tidy_sentence("Going back to school is not identical with giv...", set(model.index_to_key))
np.mean([model[word] for word in sentence], axis=0).shape,  # to average vectors

In [ ]:
np.mean([model[word] for word in sentence], axis=0);

generate vectors for first 1000 comments

In [ ]:
def save_sentence_vector(row, np_array):
    sentence = tidy_sentence(row['comment'], set(model.index_to_key))
    w2v_vectors[row.name] = np.mean([model[word] for word in sentence], axis=0)

Слишком медленно :(

In [ ]:
w2v_vectors = np.zeros([1000, 300])
tqdm.pandas()
train.progress_apply(save_sentence_vector, np_array=w2v_vectors, axis=1)

In [ ]:
first  = str(list(train.query('score == 0').head(20).values[2]))
second  = str(list(train.query('score == 4').head(20).values[2]))

compute_sentence_similarity(first, second ,model)

In [ ]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 0').head(100).values[i]))
    second  = str(list(train.query('score == 4').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

In [ ]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 0').head(100).values[i]))
    second  = str(list(train.query('score == 0').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

In [ ]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 1').head(100).values[i]))
    second  = str(list(train.query('score == 1').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

In [ ]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 2').head(100).values[i]))
    second  = str(list(train.query('score == 2').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

In [ ]:
similarity = 0;
for i in range(0, 100):
    first  = str(list(train.query('score == 3').head(100).values[i]))
    second  = str(list(train.query('score == 3').tail(100).values[i]))
    similarity += compute_sentence_similarity(first, second ,model)
similarity / 100

## TFIDF

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = train['comment'].head(100000).values # 440535
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(corpus)
print(X.shape)

(100000, 10000)


Скоры слов ????

from: https://stackoverflow.com/questions/53294482/how-to-get-tf-idf-scores-for-the-words

In [ ]:
get_word_scores = False

if get_word_scores:
    tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
    # sort in descending order
    tfidf_dict = dict(sorted(tfidf_dict.items(), key=lambda x: x[1], reverse=True))

    # initialise dataframe
    tfidf_df = pd.DataFrame.from_dict(tfidf_dict, orient='index', columns=['tfidf'])
    # name the index
    tfidf_df.index = tfidf_df.index.rename('token')
    # display first 5 rows
    tfidf_df.head()

In [85]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
svd = TruncatedSVD(n_components=2048, n_iter=21, random_state=42)
svd.fit(X)
# print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum()) # 0.68203 cleaned 0.685499

0.6854998702162306


In [86]:
X_svd = svd.transform(X)
y = train['score'].head(100000).values

knn

In [87]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 9)
split = splitter.split(X_svd, groups=train['text_id'].head(100000))
train_inds, test_inds = next(split)

X_train = X_svd[train_inds]
X_test = X_svd[test_inds]
y_train = scores[train_inds]
y_test = scores[test_inds]


neigh = KNeighborsClassifier(n_neighbors=12, metric='cosine', n_jobs=4)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
y_pred_proba = neigh.predict_proba(X_test)
accuracy_score(y_test, y_pred) # 0.2442 (svd - 2048, 21) knn: 12 # cleaned 0.24045

0.24045954045954046

accuracy по 0 - классу

In [88]:
accuracy_score(y_test[np.where(y_test == 0)],  y_pred[np.where(y_test == 0)]) # 0.61075 # cleaned 0.6048

0.6048951048951049

ndcg, надеюсь правильная? Можно наверное все значения сразу передавать в ndcg_score, без цикла

In [90]:
ndcg = 0
shift = 5

for i in tqdm(range(y_test.shape[0] //5)):
    n_values = 5
    y_test_ = np.eye(n_values)[y_test[i*5:shift]]
    ndcg += ndcg_score(y_test_, y_pred_proba[i*5:shift])
    shift += 5

ndcg = ndcg / (y_test.shape[0] // 5)
print(ndcg)

100%|██████████| 4004/4004 [00:02<00:00, 1496.26it/s]

0.6162615380158982


0.6180268727166089

Ниже не нужно

In [ ]:
!pip install torchwordemb

In [129]:
import torch
import torchwordemb # Load pretrained word embeddings (word2vec, glove format) into torch.FloatTensor for PyTorch

In [133]:
vocab, vec = torchwordemb.load_word2vec_bin("/kaggle/input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin")
print(vec.size())
print(vec[model.key_to_index["apple"]])

torch.Size([3000000, 300])
tensor([-0.0645, -0.1602, -0.0121,  0.1348, -0.2295,  0.1621,  0.3047, -0.1797,
        -0.1211,  0.2539, -0.0143, -0.0640, -0.0806, -0.0569, -0.1963,  0.2891,
        -0.0515,  0.1426, -0.1050, -0.0474, -0.3477,  0.3574,  0.2656,  0.0019,
        -0.0159,  0.0020, -0.3555,  0.2217,  0.0576,  0.1592,  0.0869, -0.0267,
        -0.0479,  0.2393, -0.0598,  0.0378,  0.1738, -0.4180,  0.2891,  0.3262,
         0.0243, -0.0165, -0.0649, -0.0889,  0.0767, -0.1514,  0.0525, -0.0420,
        -0.0542,  0.0011, -0.2012,  0.1230,  0.0923,  0.1045, -0.0041, -0.0420,
         0.0141, -0.0211, -0.1348, -0.2432,  0.1602, -0.0669, -0.0898, -0.0718,
        -0.0060, -0.0048, -0.0009, -0.3066, -0.0625,  0.0796, -0.0091, -0.0449,
         0.0996, -0.3340, -0.3984,  0.0554, -0.0669, -0.0447,  0.1177, -0.1396,
        -0.2637,  0.1748, -0.1738, -0.4062, -0.0674, -0.0762,  0.0942,  0.2100,
        -0.1631, -0.0869, -0.0535, -0.1035, -0.0762, -0.1108, -0.0352, -0.1494,
         0.03